# Coursera | Andrew Ng (05-week1)—循环序列模型

【第 5 部分-序列模型-第一周】在吴恩达深度学习视频基础上，笔记总结，添加个人理解。- ZJ
    
>[Coursera 课程](https://www.coursera.org/specializations/deep-learning) |[deeplearning.ai](https://www.deeplearning.ai/) |[网易云课堂](https://mooc.study.163.com/smartSpec/detail/1001319001.htm)

[CSDN]()：

---


# 循环序列模型 （Recurrent Neural Networks）


## <font color=#0099ff>1.1 为什么选择序列模型？(Why sequence models?)
    
深度学习中最激动人心的领域之一：序列模型。

例如循环神经网络（RNN）在语音识别，自然语言处理等其他领域中引起变革。

能够应用序列模型的例子:语音识别、音乐生成、情感分类、DNA 序列分析、机器翻译、视频活动识别、名称实体识别等。

![这里写图片描述](http://img.blog.csdn.net/20180224155604642?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
   
**学习目标：学习构建自己的序列模型。**    
    
---
## <font color=#0099ff>1.2 数学符号 (Notation)
    
从**定义数学符号**开始，一步步构建序列模型。    

![这里写图片描述](http://img.blog.csdn.net/20180225121806505?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

输入 x : Harry Potter and Hermione Granger invented a new spell.

**目标：构建**一个能够**自动识别句中人名位置**的**序列模型**。---命名实体识别问题（Named-entity recognition）NER

此例子中为：识别  Harry Potter 和 Hermione Granger 

$x^{<1>}  x^{<2>}   x^{<3>}   ......x^{<t>} ......x^{<9>} $   9 组特征表示上述 x 中 9 个单词。

输出 y : 1    1    0  1   1  0   0    0  0

$y^{<1>}  y^{<2>}   y^{<3>}   ......y^{<t>} ......y^{<9>} $ 

t : 意味着这些是时间序列。

输入序列的长度：$T_x=9$

输出序列的长度：$T_y=9$

$x^{(i)<t>}$: 第  i  个 训练样本 x 的第 t  个元素。 $T^{(I)}_x$ : 第 i 个训练样本的输入序列长度。

$y^{(i)<t>}$: 第  i  个训练样本  y 的第 t  个元素。 $T^{(I)}_y$ : 第 i 个训练样本的输出序列长度。


![这里写图片描述](http://img.blog.csdn.net/20180225121824703?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
   
 使用**自然语言处理 （NLP）**,**如何在序列中表示单个单词?**

首先，需要词汇表（字典）。列出在表示中所使用的所有单词。如图中所表示，不同的单词在自己的词汇表中对应得索引不同。
此例子中使用的 10000 个单词的词汇表，根据不同的商业模式，应用，所构建的词汇表不同，30000，50000，（大型互联网）1000000等。

**One-hot 向量表示法：**

如上图所示：$x^{<1>}$ 是一个 10000 维度的向量，只有 第 4075 索引位置是 1，其余位置都是 0 。    
    
对于不在词汇表中的单词的表示：返回   `<UNK>`    (Unknown Word)

---
## <font color=#0099ff>1.3 循环神经网络模型 (Recurrent Neural Network Model)   
 
对于上述 命名实体识别问题 (NER) 之所以不使用标准神经网络来解决，是因为存在两个主要问题。

![这里写图片描述](http://img.blog.csdn.net/20180225153812706?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 

- 1.首先是在不同的例子中输入和输出可以是不同的长度。
- 2.像这样一个简单的神经网络架构，它不会共享在文本中不同位置上学到的特征。(例如卷积神经网络，可以传递学习到的特征等)
- 此外，每一个训练样本都是一个 10,000 维的向量。所以，这只是一个非常大的输入层，同时，第一层的权重矩阵将最终具有大量的参数。

 
![这里写图片描述](http://img.blog.csdn.net/20180225153839261?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

使用**循环神经网络（RNN）** 可以解决上述两个问题。

什么是循环神经网络？

如果你从左到右阅读句子，第一个单词 $x^{<1>}$ 以及 初始化 $a^{<0>}$ 零向量 ，带入到神经网络层（隐含层），尝试预测输出 $\hat{y}^{<1>}$，同时在时间步 1 产生  $a^{<1>}$。然后，将 $a^{<1>}$ 和 $x^{<2>}$带入到神经网络层（隐含层），尝试预测输出 $\hat{y}^{<2>}$，依次循环......直到最后输入 $x^{<T_x>}$,输出 $\hat{y}^{<T_y>}$。因此，在每个时间步，循环神经网络将这个激活传递给下一个时间步，供它使用。

在这个例子中，$T_x = T_y$ ,若 $T_x ， T_y$  不同，体系结构会改变一点。

上图中右侧图表，是在 论文或书籍中常见的图表，为了便于理解，Andrew Ng 使用左侧图表展开讲解。

每个时间步的参数是共享的，每一部分都受到参数的影响，如 $W_{aa},W_{ax},W_{ya}$,后面在解释这些参数的工作原理。

**RNN 的一个弱点：**，它只使用序列中较早的信息进行预测，特别是在预测  $\hat{y}^{<3>}$ 时，不使用 $x^{<4>}，x^{<5>}，x^{<6>}$ 等单词的信息。

例如： 

1. He said,"Teddy Roosevelt was a great President."
2. He said,"Teddy bears are on sale!"

上述例子中，Teddy Roosevelt 是人名。Teddy bears 并非我们想要识别的内容，但是如果只知道前三个单词 是无法准确识别的，还需要后面的单词信息。所以这个特定的神经网络结构的一个限制是，在某一时刻的预测使用输入或使用来自序列中较早的输入的信息，而不是在该序列的后面的信息。

后面的视频将讲解 **双向递归神经网络 （BRNN）**，来解决这个问题。

![这里写图片描述](http://img.blog.csdn.net/20180225153859865?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**前向传播：** 

$a^{<0>}=\vec{0}$  

$a^{<1>}=g(W_{aa}a^{<0>} + W_{ax}x^{<1>} + b_a)$ 
$\hat{y}^{<1>} = g(W_{ya}a^{<1>} + b_y)$

$a^{<t>}=g(W_{aa}a^{<t-1>} + W_{ax}x^{<t>} + b_a)$ 
$\hat{y}^{<t>} = g(W_{ya}a^{<t>} + b_y)$

对于激活函数计算 $a^{<t>}$常用的是 **tanh** , 也有使用 **Relu** 的。

对于输出 y ,根据具体问题选取不同激活函数，如二分类问题可使用 **sigmoid**，其他 输出 如 softmax 等。

![这里写图片描述](http://img.blog.csdn.net/20180225153912301?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

**简化 RNN 符号表示：**

$a^{<t>}=g(W_{aa}a^{<t-1>} + W_{ax}x^{<t>} + b_a)$ 
$\hat{y}^{<t>} = g(W_{ya}a^{<t>} + b_y)$

简化为：

$a^{<t>}=g(W_{a}[a^{<t-1>} ,x^{<t>} ]+ b_a)$ 
$\hat{y}^{<t>} = g(W_{y}a^{<t>} + b_y)$    

使用矩阵的叠加简化，如 a 的维度是 100，x 的维度是 10000，那么 $W_{aa}$ 的维度是 （100，100），$W_{ax}$ 的维度是 （100，10000）,叠加后 $W_{a}$ 的维度是 （100，100100），如上图中所示。

    
---
## <font color=#0099ff>1.4 通过时间的反向传播  (Backpropogation through time)  
 
 上一节讲述的是 RNN 的基本结构，以及前向广播，这一节学习目标是：了解 RNN 的反向传播是如何工作的。
 
![这里写图片描述](http://img.blog.csdn.net/20180225180047463?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

红色箭头方向是反向传播。

![这里写图片描述](http://img.blog.csdn.net/20180225181207104?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSlVOSlVOX1pIQU8=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    
如上图所示：红色箭头方向是反向传播。绿色线参数相同。

逻辑回归损失，也称为交叉熵损失。

单个单一位置或单个时间单一预测的单个词汇的损失：

$L^{<t>}(\hat{y}^{<t>},y^{<t>})= - y^{(i)}log\hat{y}^{<t>} - (1- y^{<t>}) log(1-\hat{y}^{<t>} )$

如上图所示，每个特征元素计算损失值。

整个序列的整体损失：

$L(\hat{y},y)= \sum_{t=1}^{T_y} L^{<t>}(\hat{y}^{<t>},y^{<t>})$


以上是 RNN 前向和反向的工作原理，以上讲解的例子都是输入序列的长度等于输出序列的长度，接下来会展示更广泛的RNN架构。

---
## <font color=#0099ff>1.5 不同类型的循环神经网络    
    
    
---
## <font color=#0099ff>1.6 语言模型和序列生成    
    
    
    
---
## <font color=#0099ff>1.7 对新序列采样    
    
    
    
---
## <font color=#0099ff>1.8 带有神经网络的梯度消失    
    
    
---
## <font color=#0099ff>1.9 GRU 单元      
    
    
---
## <font color=#0099ff>1.10 长短期记忆 （LSTM）     
  


---
## <font color=#0099ff>1.11 双向神经网络 
    

---
## <font color=#0099ff>1.12 深层循环神经网络
    
    
    
    
---
 
**PS: 欢迎扫码关注公众号：「SelfImprovementLab」！专注「深度学习」，「机器学习」，「人工智能」。以及 「早起」，「阅读」，「运动」，「英语 」「其他」不定期建群 打卡互助活动。**

<center><img src="http://upload-images.jianshu.io/upload_images/1157146-cab5ba89dfeeec4b.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240"></center>